# The Math behind the model

### Computing Latent Semantic Relations and context word vectors using LSA

Let `D` be all the documents in the text corpus. (m number of documents $\therefore D_{1\times m}$) $\\$ 
Let `V` be the vocabulary of all words in it. (n number of words $\therefore V_{1\times n}$)

Create a term-document matrix $X_{n\times m}$ where $\\$
$\large X_{i,j} = tf(v_i, d_j) \times idf(v_i)$ 

Define: $\\\large tf(v_i, d_j) = log(1+\frac{f_{t_i,d_j}}{\sum_{t^\prime \in V}f_{t^\prime, d_j}})\\$
$\large idf(v_i) = log(\frac{|D|}{|{d \in D : t \in d}|})$

The term-frequency($tf$) function calculates the logarithmically scaled frequency of the word $v_i$ ocuurring in the document $d_j$

The inverse-document-frequency($idf$) function calculates the inverse of the probability that the word $v_i$ occurs in the word whole document `D`. This is done to negate the effect of words that are present in a huge number of documents as they hold lesser relevance.

$\large P(v/D) = \frac {|{d \in D \ : \ t \in d}|}{|D|} \\$
$\large idf = -log(P(v/D)) = log(\frac {1}{P(v/D)}) = log(\frac{|D|}{|{d \in D : t \in d}|})$



Since `n` is very large, it is difficult to gather information from this matrix X. Thus we perform Truncated Singular Value Decomposition on this matrix to get the most relevant information from the matrix.

SVD is a complex mathematical procedure of factorizing a matrix into three factor matrices.

$\large X = U\Sigma V^T$

Here $U, V^T$ are orthogonal square matrices.
$U$ is called the left singular matrix and $V^T$ is called the right singular matrix.
$\Sigma$ is the matrix of singular values at diagonal entries.

$X = \begin{bmatrix} x_{11} & x_{12} & \dots & x_{1n} \\ x_{21} & x_{22} & \dots & x_{2n} \\ \vdots & \vdots & \dots & \vdots \\ \vdots & \vdots & \dots & \vdots \\ \vdots & \vdots & \dots & \vdots\\ x_{n1} & x_{n2} & \dots & x_{nm}  \end{bmatrix} = 
\begin{bmatrix}  u_{11} & u_{12} & \dots & \dots & \dots & u_{1n} \\  u_{21} & u_{22} & \dots & \dots & \dots & u_{2n} \\ \vdots & \vdots & \dots & \dots & \dots & \vdots \\ \vdots & \vdots & \dots & \dots &\dots & \vdots\\ \vdots & \vdots & \dots & \dots &\dots & \vdots\\ u_{n1} & u_{n2} & \dots & \dots & \dots & u_{nn} \end{bmatrix}
 . \begin{bmatrix} \sigma_1 & 0 & \dots & 0  \\ 0 & \sigma_2 & \dots & 0 \\ \vdots & \vdots & \dots & \vdots \\ \vdots & \vdots & \dots & \sigma_m \\ \vdots & \vdots & \dots & \vdots \\ 0 & \dots & \dots & 0 \end{bmatrix} .
  \begin{bmatrix}  v_{11} & v_{12} & \dots & v_{1m} \\  v_{21} & v_{22} & \dots & v_{2m} \\ \vdots & \vdots & \dots & \vdots \\ \vdots & \vdots & \dots & \vdots\\ v_{m1} & v_{m2} & \dots & v_{mm} \end{bmatrix}$ 



The singular value matrix $\Sigma$ has n singular values. These singular values are in descending order. Thus if we take the first `k` singular values only, it must approximate $X$ relatively good.

$\large \therefore X^\prime = U^\prime \Sigma^\prime V^{T^\prime}$ where $\large U^\prime, \Sigma^\prime \ and \ V^{T^\prime}$ are truncated matrices

$\large U_{n\times k}^\prime$ is the the matrix of n words having k dimensional vectors.

$\large V_{k\times m}^{T^\prime}$ is the matrix of k semantic relations distributed across m documents.

#### Extracting the Topics from $V^{T^\prime}$

Let $\large K_i$ be the topic that represents the document $d_i$

Then, $\large K_i = argmax(V^{\prime}_i)$

The `argmax()` function returns the maximum entry from the $i^{th}$ column of $V^{T^\prime}$ or $i^{th}$ row from $V^{\prime}$

Thus, now we have $K_{1\times m}$ matrix of topics that represent the documents.

### Direct Semantic Relations, Word Analogies and Word Meaning computation

As we have calculated the latent semantic relations that are related through topics, we must now try to calculate the direct semantic relations between words itself. This will give us similarity between words and also word-sense.

What we intend to do is to use 2 k-dimensional word-vectors per word and modify them such that we can predict context words given the center word. This will be done through skimming the vocabulary in windows of certain width and calculating the likelihood of a word being in the context of the given center word.

Let $v_w$ be the vector when `w` is the center word.

Let $u_w$ be the vector when `w` is the context word.

Since, we already have k dimensional vectors for our words in $U^\prime$, we can use them as context vectors.

And thus define randomly initialized k dimensional vectors for center word vectors.

We are using 2 vectors so as to ease our further calculations as the self inner space product might cause complicated calculations.

Define, $\large L(\theta) = \large {\Pi_{t=1}^{T} \Pi_{-m\leq j\leq m, j\neq 0} P(w_{t+j}/w_t \ ; \ \theta)}$

where `t` is the position of the word in the corpus, `m` is the window size and $\theta$ is the variable that needs to be optimized for better results.

$\theta = \begin{bmatrix} v_{w_1} \\ v_{w_2} \\ \vdots \\ v_{w_n} \\ u_{w_1} \\ \vdots \\ u_{w_n} \end{bmatrix}$ is the parameter for training the model.

By Total Probability Theorem:

$\large P(o/c) = \Large {\frac {e^{u_o v_c}} {\sum_{w\in V} e^{u_w v_c}}}$, where `o` is the context word and `c` is the center word.

The `softmax` function has been used to keep the probability positive and within the [0,1]

The cost function for the following model is defined as:

$\large J(\theta) = -\frac{1}{T} log(L(\theta))$

The negative sign is for us to be able to minimize the cost function as we need to increase the likelihood. And division by T is to get the average out total likelihood.

Because:

$\large \frac {\partial}{\partial v_c} log({\frac {e^{u_o v_c}} {\sum_{w\in V} e^{u_w v_c}}}) = \frac {\partial}{\partial v_c} log(e^{u_o v_c}) - \frac {\partial}{\partial v_c} log(\sum_{w\in V} e^{u_w v_c})$

$\large = u_o - \frac {\partial}{\partial v_c} log(\sum_{w\in V} e^{u_w v_c})$

$\large = u_o - \frac {1}{\sum_{w\in V} e^{u_w v_c}} . \frac {\partial}{\partial v_c}\sum_{w\in V} e^{u_w v_c}$

$\large = u_o - \frac {\sum_{w\in V} u_w\times e^{u_w v_c}}{\sum_{w\in V} e^{u_w v_c}} $

Taking w = x in numerator

$\large = u_o - u_x.\sum_{x\in V}(\frac {e^{u_x v_c}}{\sum_{w\in V} e^{u_w v_c}}) $

$\large = u_o - u_x.\sum_{x\in V}P(x/c) $

= observed_vector - expected_vector

#### Minimizing the cost-function $J(\theta)$

Gradient Descent is an algorithm to minimize $J(\theta)$ by changing $\theta$

=>Calculate Gradient of $J(\theta)$

=>Take small step in the direction of negative gradient

=>Repeat

$\large \theta_{new} = \theta_{old} - \alpha \nabla J(\theta)$

where $\alpha$ is the step size and is a hyperparameter that can be varied to get appropriate result.

But for very large corpus $\nabla J(\theta)$ is very expensive to compute

Thus we use Stochastic Gradient Descent to repeatedly sample windows and update after each small batch.

In the softmax probability the normalizing term $\large \sum_{w\in V} e^{u_wv_c}$ is expensive to compute, we can also use negative sampling.

$\large J(\theta) = - \frac{1}{T} \sum_{t=1}^T J_t(\theta)$

where, $\large J_t(\theta) = log(\sigma(u_ov_c)) + \sum_{j \sim P(w)}log(\sigma(-u_jv_c))$

Here, $\large \sigma(x) = \frac {1}{1+e^{-x}}$ is the sigmoid function that keeps the value between [0,1]

We sample the random words based on their frequency occurrence. $P(w) = {U(w)^{\frac{3}{4}}}$, where $U(w)$ is the unigram distribution of the whole corpus, in which words are distributed as per their ocurrences in the corpus $\therefore U(w) = f_{w,D}$. The 3/4 power is to make the words with low ocurrences also get sampled.

The first term with $\sigma (u_ov_c)$ is to maximize the probability when two words co-occur. While the second term with the negative sign $\sigma (-u_jv_c)$ is to minimize the probability for the noise words.

#### Meaning Component in Word Vectors

After completing the stochastic gradient descent, we get the improved word vectors in $\theta$

To analyze meanings from these word vectors we can use cosine similarity to calculate similar words with similar usages/meanings.

$\large cos(\theta) = \frac {v_1.v_2}{||v_1||.||v_2||}$

$\large \therefore w_m = argmax_i(\frac {v_w.v_{w_i}}{||v_w||.||v_{w_i}||})$, where $v_w$ is the vector for the word for which the meaning component is to be calculated and $v_{w_i}$ are the vector for the words in the vocabulary

#### Word Analogies Computation

Word Analogies can be computed using the meaning component of the word vectors.

Let $w_a:w_b::w_c:(w_d)?$, then $\large w_d = argmax_i \frac{((v_{w_b}-v_{w_a}+v_{w_c}).v_{w_i})}{|v_{w_b}-v_{w_a}+v_{w_c}|.|v_{w_i}|}$

Here we are doing vector addition. Taking an example, if man : king :: woman : ?. Then we remove the vector of man fromm the vector of king and add the vector of woman to it.($v_{king} - v_{man} + v_{woman}$) and check which word is closest to meaning to that.